<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js041_windTunnelSPH2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [1]:
#@title js041_windTunnelSPH2D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023-2024 Mitsuru Ikeuchi
#  home page: https://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.09.29 created,  last updated on 2024.05.18
#

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js041_windTunnelSPH2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js041_windTunnelSPH2D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.06.15 created, last updated on 2018.11.19
//    ver 0.0.1  2019.01.17 v1, last updated on 2021.06.01
//    ver 0.0.2  2021.11.01 v2, last updated on 2021.11.01
//    ver 0.0.3  2023.03.22 v3, last updated on 2023.08.24
//
// --------------------  smoothed particle hydrodynamics 2D
//
// - particle base Lagrangian method
//    Monaghan; "Smoothed Particle Hydrodynamics"
//              Annu. Rev. Astron. Astrophys.1992. 30:543-74
//
//    W(x-xi,h) : kernel weight function (q=|x-xi|/h)
//      = aKernel*(1-1.5*q^2+0.75*q^3)  (q<1)
//      = aKernel*0.25*(2-q)^3  (1<=q<2)
//      = 0  (q>=2)
//
//        aKernel = 2/3/h (1D)
//                = 10/(7Pi)/h^2 (2D)
//                = 1/Pi/h^3 (3D)
//
//    f(x) --> sum[mj/rhoj*f(xj)*W(x-xj,h), j]
//    grad f(x) --> -sum[mj/rhoj*f(xj)*grad W(x-xj,h), j]
//
// - time step
//   (1) registration - set section[][][]
//   (2) set density[]
//   (3) set pressure[]
//   (4) Verlet step1 (t = t + dt/2)
//   (5) set acceleration ax[] ay[] and power[] <-- x(t+dt/2),y(t+dt/2)
//   (6) Verlet step2 (t = (t + dt/2) + dt/2)
//   (7) set boundary
//   goto (1)
//
// - for fast calculation - O(N) calculation
//     set section: smoothed particle divided into lattice section (see setSection() )
//     calculate acceleration: sum up near section (see accCalc() )
//       near means r<2*hh0
//
// - artificial viscosity
//     dv_a/dt = -sum(P_b/rho_b^2 + P_a/rho_a^2 + pai_ab) grad W_ab
//       pai_ab = (-alpha c_ab mue_ab + beta mue_ab^2)/rho_ab (v_ab*r_ab<0) or 0 (v_ab*r_ab>0)
//         mue_ab = (h v_ab*r_ab)/(r_ab^2+etha^2), etha^2 nearly= 0.01*hh0*hh0
//
// --------------------
*/

const windTunnelSPH2D = (function(){ // ====================  windTunnelSPH2D Module  ====================

	let g_NNp = 5000;						// number of smoothed particles
	let g_Nsx = 40;							// number of division to x-lattice section
	let g_Nsy = 20;							// number of division to y-lattice section
	let g_xMax = 20.0;						// (m) x-Box size
	let g_yMax = 10.0;						// (m) y-Box size
	let g_sysTime = 0.0;					// (s) system time
	let g_dt = 0.0001;						// (s) time step
	let g_hh0 = 0.25;						// (m) diameter of smoothed particles
	let g_rBond = g_hh0*1.2;				// (m) bonding upper limit
	let g_cViscosity = 1.00;				// for air viscosity, alpha=1,beta=2 in this case
	let g_thermalDiffusivity = 0.1;			// (m^2/s) thermal diffusivity eg. 3.4e-5
	let g_gravity = -9.8;					// (m/s^2) gravity acceleration eg. 9.8;
	let g_vx0 = 50.0;						// (m/s) fan flow speed
	let g_temp0 = 300.0;					// (K) reference temperature

	const g_kind = dimInt1( g_NNp );		// smooth particle kind 1:air 0:wood-fixed -1:wood-free
	const g_bond = dimInt2( g_NNp, 8 );		// bonded particle No. with i-th particle
	const g_xx = dim1( g_NNp );				// (m) smooth particle pos-x
	const g_yy = dim1( g_NNp );				// (m) smooth particle pos-y
	const g_vx = dim1( g_NNp );				// (m/s) smooth particle velocity-x
	const g_vy = dim1( g_NNp );				// (m/s) smooth particle velocity-y
	const g_ax = dim1( g_NNp );				// (m/s^2) smooth particle total accelaration-x
	const g_ay = dim1( g_NNp );				// (m/s^2) smooth particle total accelaration-y
	const g_hh = dim1( g_NNp );				// (m) smooth particle diameter
	const g_mass = dim1( g_NNp );			// (kg) smooth particle mass
	const g_cp = dim1( g_NNp );				// (J/(kg*K)) smooth particle specific heat
	const g_density = dim1( g_NNp );		// (kg/m^3) smooth particle density
	const g_pressure = dim1( g_NNp );		// (N/m^2) smooth particle pressure
	const g_energy = dim1( g_NNp );			// (J) smooth particle energy = mass*cp*temp
	const g_power = dim1( g_NNp );			// (W) smooth particle power : energy(t+dt) = energy(t)+power*dt
	const g_section = dimInt3( g_Nsx, g_Nsy, 50 ); // registration: particles in Nsx x Nsy sections

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dimInt1( n ) {
		return new Int32Array( n );
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}

	function dimInt3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] =  new Int32Array( nk );
			}
		}
		return a;
	}


	// --------------------  set initial condition  --------------------

	// public
	function setInitialCondition() {
		g_sysTime = 0.0;
		g_NNp = setParticles(0);

		bonding();
		setSection();
		setDensity();
		for (let i=0; i<g_NNp; i++) {
			g_energy[i] = g_mass[i]*g_cp[i]*g_temp0;
			g_power[i] = 0.0;
		}
	}

	function setParticles(i) {
		const nx = Math.floor(g_xMax/0.3);
		const ny = Math.floor((g_yMax-g_hh0*2)/0.3/0.866);
		for (let ix=0; ix<nx; ix++) {
			for (let iy=0; iy<ny; iy++) {
				const x = 0.25+0.3*(ix)+0.15*(iy%2);
				const y = 0.3+0.3*(0.866*iy);
				if ((x>6.0 && x<6.8) && (y>3.5 && y<6.5)) { // wood
					g_kind[i] = 0; // fixed particle
					g_mass[i] = 300.0/4.0;
					g_cp[i] = 1500.0;
					g_xx[i] = x;
					g_yy[i] = y;
					g_vx[i] = 0.0;
					g_vy[i] = 0.0;
					g_ax[i] = 0.0;
					g_ay[i] = 0.0;
					g_hh[i] = g_hh0;
					i += 1;
				} else { // air
					g_kind[i] = 1; // SPH particle
					g_mass[i] = (0.029/0.0224)/4.0;
					g_cp[i] = 1000.0;
					g_xx[i] = x;
					g_yy[i] = y;
					g_vx[i] = 0.5*(Math.random()-0.5)+g_vx0;
					g_vy[i] = 0.5*(Math.random()-0.5);
					g_ax[i] = 0.0;
					g_ay[i] = 0.0;
					g_hh[i] = g_hh0;
					i += 1;
				}
			}
		}
		return i;
	}


	// --------------------  time evolution  --------------------

	// public
	function timeEvolution( nCalc ) {
		for (let i=0; i<nCalc; i++) {
			timeStep();
			g_sysTime += g_dt;
		}
	}

	function timeStep() {
		const nnp=g_NNp, dtv2=g_dt/2.0;

		// (1) registration - set section[]
		setSection();

		// (2) set density
		setDensity();

		// (3) set pressure
		for (let i=0; i<nnp; i++) {
			if (g_kind[i]>0) {
				g_pressure[i] = 8.31*g_density[i]*(g_energy[i]/(g_mass[i]*g_cp[i]));
			} else {
				g_pressure[i] = 0.5*g_density[i];
			}
		}

		// (4) Verlet step1 (t = t + dt/2)
		for (let i=0; i<nnp; i++) {
			if (g_kind[i]==0) continue;
			g_vx[i] += dtv2*g_ax[i];
			g_vy[i] += dtv2*g_ay[i];
			g_xx[i] += g_vx[i]*g_dt;
			g_yy[i] += g_vy[i]*g_dt;
		}

		// (5) set acceleration ax[] ay[] and power[]
		accCalc();

		// (6) Verlet step2 (t = t + dt/2 + dt/2)
		for (let i=0; i<nnp; i++) {
			if (g_kind[i]==0) continue;
			g_vx[i] += dtv2*g_ax[i];
			g_vy[i] += dtv2*g_ay[i];
			g_energy[i] += g_power[i]*g_dt;
		}

		// (7) boundary
		const rr = 1.0;
		for (let i=0; i<nnp; i++) {
			if (g_xx[i] < 0.0) {
				g_xx[i] += g_xMax;
			}
			if (g_xx[i] > g_xMax) {
				g_xx[i] -= g_xMax;
			}
			if (g_yy[i] < 0.0) {
				g_yy[i] = 0.0; g_vx[i] = rr*g_vx[i]; g_vy[i] = -rr*g_vy[i];
			}
			if (g_yy[i] > g_yMax) {
				g_yy[i] = g_yMax; g_vx[i] = rr*g_vx[i]; g_vy[i] = -rr*g_vy[i];
			}
		}

		// fan
		for (let i=0; i<nnp; i++) {
			if (g_xx[i] < 0.3) {
				g_energy[i] = g_mass[i]*g_cp[i]*g_temp0;
				g_vx[i] = g_vx0;
			}
		}
	}

	//--- (1) registration

	function setSection() {
		const nnp=g_NNp, nsx=g_Nsx, nsy=g_Nsy;
		for (let i=0; i<nsx; i++) {
			for (let j=0; j<nsy; j++) {
				g_section[i][j][0] = 0;
			}
		}
		for (let ip=0; ip<nnp; ip++) {
			let i = Math.floor(nsx*g_xx[ip]/g_xMax); if (i>=nsx) i = nsx-1;
			let j = Math.floor(nsy*g_yy[ip]/g_yMax); if (j>=nsy) j = nsy-1;
			const iq = g_section[i][j][0]+1;
			g_section[i][j][0] = iq;
			g_section[i][j][iq] = ip;
		}
	}

	function maxSection() {
		const nsx=g_Nsx,nsy=g_Nsy;
		let m=0;
		for (let i=0; i<nsx; i++) {
			for (let j=0; j<nsy; j++) {
				if (g_section[i][j][0]>m) m = g_section[i][j][0];
			}
		}
		return m;
	}

	//--- (2) set density

	function setDensity() {
		const nnp=g_NNp, nsx=g_Nsx, nsy=g_Nsy;
		const rcc = g_hh0*2.0;
		const rcc2 = rcc*rcc;
		for (let i=0; i<nnp; i++) {
			let s = 0.0;
			let i0 = Math.floor(nsx*(g_xx[i]-rcc)/g_xMax+nsx)-nsx;
			let i1 = Math.floor(nsx*(g_xx[i]+rcc)/g_xMax);
			let j0 = Math.floor(nsy*(g_yy[i]-rcc)/g_yMax+nsy)-nsy;
			let j1 = Math.floor(nsy*(g_yy[i]+rcc)/g_yMax);
			for (let ii=i0; ii<=i1; ii++) {
				const iii = (ii+nsx)%nsx;
				const ikind = g_kind[iii];
				for (let jj=j0; jj<=j1; jj++) {
					const jjj = (jj+nsy)%nsy;
					for (let iq=1; iq<=g_section[iii][jjj][0]; iq++) {
						const j = g_section[iii][jjj][iq];
						if (g_kind[j]==ikind) {
							let xij = g_xx[i] - g_xx[j], yij = g_yy[i] - g_yy[j];
							if (xij>0.5*g_xMax) xij -= g_xMax;
							if (xij<-0.5*g_xMax) xij += g_xMax;
							if (yij>0.5*g_yMax) yij -= g_yMax;
							if (yij<-0.5*g_yMax) yij += g_yMax;
							const r2 = xij*xij+yij*yij;
							if (r2<rcc2) {
								const rij = Math.sqrt(r2);
								s += g_mass[j]*kernel(rij, (g_hh[i]+g_hh[j])/2.0);
							}
						}
					}
				}
			}
			g_density[i] = s;
		}
	}

	//--- (5) set acceleration ax[] ay[] and power[]

	function accCalc() { // x-periodic, y-periodic but y-boundaryForce
		const nnp=g_NNp, nsx=g_Nsx, nsy=g_Nsy;
		const alpha=1.0, beta=2.0, cv = 400.0;  // for viscosity
		const rcc = g_hh0*2.0;
		const rcc2 = rcc*rcc;
		for (let i=0; i<nnp; i++) {
			g_ax[i] = 0.0; g_ay[i] = g_gravity;
			boundaryForce(i, rcc, 1000000.0);
			g_power[i] = 0.0;
			const ai = g_pressure[i]/(g_density[i]*g_density[i]);

			let i0 = Math.floor(nsx*(g_xx[i]-rcc)/g_xMax+nsx)-nsx;
			let i1 = Math.floor(nsx*(g_xx[i]+rcc)/g_xMax);
			let j0 = Math.floor(nsy*(g_yy[i]-rcc)/g_yMax+nsy)-nsy;
			let j1 = Math.floor(nsy*(g_yy[i]+rcc)/g_yMax);
			for (let ii=i0; ii<=i1; ii++) {
				const iii = (ii+nsx)%nsx;
				for (let jj=j0; jj<=j1; jj++) {
					const jjj = (jj+nsy)%nsy;
					for (let iq=1; iq<=g_section[iii][jjj][0]; iq++) {
						let j = g_section[iii][jjj][iq];

						let xij = g_xx[i] - g_xx[j], yij = g_yy[i] - g_yy[j];
						if (xij>0.5*g_xMax) xij -= g_xMax;
						if (xij<-0.5*g_xMax) xij += g_xMax;
						if (yij>0.5*g_yMax) yij -= g_yMax;
						if (yij<-0.5*g_yMax) yij += g_yMax;
						const r2=xij*xij+yij*yij;
						if (r2<rcc2) {
							const rij = Math.sqrt(r2);
							if (rij>0) {
								if (g_kind[j]>0 && g_kind[i]>0) {
									const aj = g_pressure[j]/(g_density[j]*g_density[j]);
									const vr = xij*(g_vx[i]-g_vx[j])+yij*(g_vy[i]-g_vy[j]); // for viscosity
									let paiij = 0;
									if (vr<0.0) { // for viscosity
										const mu = 0.5*vr/(r2+0.01*g_hh0*g_hh0);
										paiij = (-alpha*cv*mu+beta*mu*mu)/((g_density[i]+g_density[j])/2.0);
									}
									const b = g_mass[j]*(ai+aj+paiij); // for viscosity
									const gradW = dwwvdr(rij, (g_hh[i]+g_hh[j])/2.0);
									const gradWx = gradW*xij/rij;
									const gradWy = gradW*yij/rij;
									g_ax[i] += -b*gradWx;
									g_ay[i] += -b*gradWy;
									const lapW = d2wwvdr2(rij, (g_hh[i]+g_hh[j])/2.0);
									g_power[i] += 0.5*b*((g_vx[i]-g_vx[j])*gradWx+(g_vy[i]-g_vy[j])*gradWy) +
												g_thermalDiffusivity*g_mass[j]/g_density[j]*(g_energy[j]-g_energy[i])*lapW;
								} else {
									const a = force(rij,i,j)/g_mass[i];
									g_ax[i] += a*xij/rij;
									g_ay[i] += a*yij/rij;
								}
							}
						}
					}
				}
			}
		}
	}

	// called only if (ii or jj is not SPH particle)
	function force(r, ii, jj) {
		if (g_kind[ii]>=0 || g_kind[jj]>=0) {
			const rc = g_hh0*1.2;
			let ret = 0.0;
			if (r<rc) {
				const ra = (rc-r)/rc;
				ret = 50000000.0*ra*ra;
				return ret;
			}
		}

		if( g_kind[ii]<0 && g_kind[jj]<0 ) {
			const kc = 50000000.0;
			const ra = g_rBond;
			const ra1 = 1.2*ra;
			const ra2 = 1.5*ra;
			const ram = (ra1+ra2)/2.0;
			const fa = 2.0/((ra2-ra1)*(ra2-ra1));
			let i, j;
			if (ii<jj) {
				i= ii; j=jj;
			} else {
				i= jj; j=ii;
			}
			if (r>ra2) {
				bondUnreg(i,j); bondUnreg(j,i);
				return 0.0;
			}
			let f = 0.0;
			if (bondQ(i,j)==0) {
				if (r<ra) {
					f = kc*(r-ra);
				}
			} else {
				if (r<=ra1) {
					f = kc*(r-ra);
				} else if (r>=ra2) {
					f = 0.0;
					bondUnreg(i,j); bondUnreg(j,i);
				} else if (ra1<=r && r<ram) {
					f = kc*(r-ra)*(1.0-fa*(r-ra1)*(r-ra1));
				} else if (ram<=r && r<ra2) {
					f = kc*(r-ra)*fa*(r-ra2)*(r-ra2);
				}
			}
			return -f;
		}
		return 0.0;
	}

	function boundaryForce(i, rc, k) {
		let r
		if (g_yy[i] < rc) {
			r = rc-g_yy[i];
			g_ay[i] += k*r*r;
		}
		if (g_yy[i] > g_yMax-rc) {
			r = g_yy[i]-(g_yMax-rc);
			g_ay[i] += -k*r*r;
		}
	}

	//--- bonding manager

	function bonding() {
		const nnp=g_NNp, rb = g_rBond*1.2;
		const rb2=rb*rb;
		bondInit();
		for (let i=0; i<nnp; i++) {
			if (g_kind[i]<=0) {
				for (let j=i+1; j<nnp; j++) {
					if (g_kind[j]<=0) {
						const r2 = (g_xx[i]-g_xx[j])*(g_xx[i]-g_xx[j])+(g_yy[i]-g_yy[j])*(g_yy[i]-g_yy[j]);
						if (r2<=rb2) {
							bondReg(i, j); bondReg(j, i);
						}
					}
				}
			}
		}
	}

	function bondInit() {
		const nnp=g_NNp;

		for (let i=0; i<nnp; i++) {
			for (let j=0; j<8; j++) {
				g_bond[i][j]=-1;
			}
		}
	}

	function bondReg(ii, jj) {
		for (let j=0; j<8; j++) {
			if (g_bond[ii][j]==-1) {
				g_bond[ii][j] = jj;
				break;
			}
		}
	}

	function bondUnreg(ii, jj) {
		for (let j=0; j<8; j++) {
			if (g_bond[ii][j]==jj) {
				g_bond[ii][j] = -1;
				break;
			}
		}
	}

	function bondQ(ii, jj) {
		const rb = g_rBond*1.2;
		const rb2=rb*rb;
		let q = 0;
		for (j=0; j<8; j++) {
			if (g_bond[ii][j]==jj) {
				q = 1;
				break;
			}
		}
		return q;
	}

	//--- smoothed particle

	function kernel(r, h) {
		const q = r/h, a = 10.0/(7.0*3.141592)/(h*h);
		let ret = 0.0;
		if (q<1.0) {
			ret = a*(1.0-1.5*q*q+0.75*q*q*q);
		} else if (q<2.0) {
			ret = a*0.25*(2.0-q)*(2.0-q)*(2.0-q);
		}
		return ret;
	}

	function dwwvdr(r, h) {
		const q = r/h, a = 10.0/(7.0*3.141592)/(h*h*h);
		let ret = 0.0;
		if (q<1.0) {
			ret = a*(-3.0*q+2.25*q*q);
		} else if (q<=2.0) {
			ret = -a*0.75*(2.0-q)*(2.0-q);
		}
		return ret;
	}

	function d2wwvdr2(r, h) {
		const q = r/h, a = 10.0/(7.0*3.141592)/(h*h*h*h);
		let ret = 0.0;
		if (q<1.0) {
			ret = a*(-3.0+4.5*q);
		} else if (q<=2.0) {
			ret = a*0.75*(2.0-q);
		}
		return ret;
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition()
		evolve:			timeEvolution,			// timeEvolution( nCalc )

		getSysParam:	function() { return [ g_xMax, g_yMax, g_dt, g_hh0, g_NNp ]; },
		getNow:			function() { return [ g_sysTime, g_gravity ]; },
		getVx0:			function() { return g_vx0; },
		getSPKind:		function(i) { return g_kind[i]; },
		getSPData:		function(i) { return [ g_hh[i], g_mass[i], g_cp[i] ]; },
		getPosition:	function(i) { return [ g_xx[i], g_yy[i] ]; },
		getVelocity:	function(i) { return [ g_vx[i], g_vy[i] ]; },
		getAcc:			function(i) { return [ g_ax[i], g_ay[i] ]; },
		getDensity:		function(i) { return g_density[i]; },
		getPressure:	function(i) { return g_pressure[i]; },
		getTemp:		function(i) { return (g_energy[i]/(g_mass[i]*g_cp[i])); },
		getEnergy:		function(i) { return g_energy[i]; },
		getPower:		function(i) { return g_power[i]; },
	};

})(); // ====================  windTunnelSPH2D end  ====================


const js041 = (function(){ // ====================  js Module  ====================

	const theModule = windTunnelSPH2D;
	const xCanvasSize = 540;	// in pixel
	const yCanvasSize = 360;	// in pixel
	let canvas;
	let ctx;

	let v_theme = 0;  // no use
	let v_nCalc = 2;

	let p_xMax, p_yMax, p_dt, p_hh0, p_NNp; // = theModule.getSysParam();
	let sysTime, gravity;
	let nowData = [];
	let kindList = [];
  let xxList = [];
	let yyList = [];
	let vxList = [];
	let vyList = [];
  let densList = [];
	let pressList = [];
	let tempList = [];

	let dispMode = 4;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;

	let breakFlag = false;
	let perticleFlag = true;
	let timeStamp = 0.0;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( );
			[ p_xMax, p_yMax, p_dt, p_hh0, p_NNp ] = theModule.getSysParam();
			perticleFlag = true;
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_nCalc );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( 1 );
		}

		draw( ctx, dispMode );

		if ( perticleFlag ) setParticlesData();

		requestAnimationFrame(animate);
	}

	function setParticlesData() {
		nowData = [ sysTime, p_NNp ];
    for (let i=0; i<p_NNp; i++) {
			let x, y, vx, vy;
			[ x, y ] = theModule.getPosition(i);
			xxList[i] = x;
			yyList[i] = y;
			[ vx, vy ] = theModule.getVelocity(i);
			vxList[i] = vx;
			vyList[i] = vy;
			kindList[i] = theModule.getSPKind(i);
			densList[i] = theModule.getDensity(i);
			pressList[i] = theModule.getPressure(i);
			tempList[i] = theModule.getTemp(i);
		}
	}


	// --------------------  draw  --------------------

	function draw( ctx, dispMode ) {
		const xp = 20, yp = 20, xSize = 500, ySize = 250, scale = xSize/p_xMax;

		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		ctx.strokeStyle = "#888800";
		ctx.strokeRect( xp, yp, xSize, ySize );

		if ( dispMode<=3 ) {
			drawField( ctx, dispMode, p_NNp, p_hh0, scale, xp, yp );
		} else if ( dispMode==4 ) {
			drawFlow( ctx, p_NNp, scale, xp, yp );
		}

		// caption
		[ sysTime, gravity ] = theModule.getNow();
		ctx.fillStyle = "#888888";
		ctx.fillText(`Box = ${p_xMax.toFixed(1)}x${p_yMax.toFixed(1)} (m)`, xp, yCanvasSize-30);
		ctx.fillText(`N = ${p_NNp}`, xp+260, yCanvasSize-30);
		ctx.fillText(`time = ${sysTime.toFixed(3)} (s)`, xp, yCanvasSize-10);
		//document.getElementById("text_caption").innerHTML = msg;
	}

	function drawField( ctx, dispMode, nnp, hh0, sc, xp, yp ) {
		const yMax = p_yMax, twoPi = 2.0*Math.PI, dmag=5.0, pmag=0.003, tmag=2.0;

		for (let i=0; i<nnp; i++) {
			const kind = theModule.getSPKind(i);
			let xi, yi, hue;
			[ xi, yi ] = theModule.getPosition(i);
			if ( kind==1 ) { // 1:air
				if (dispMode==0) { // smooth particle
					hue = 180;
				} else if (dispMode==1) { // density
					const dens = theModule.getDensity(i);
					hue = Math.floor(18180-dens*dmag) % 360;
				} else if (dispMode==2) { // pressure
					const press = theModule.getPressure(i);
					hue = Math.floor(18180-press*pmag) % 360;
				}  else if (dispMode==3) { // temp
					const temp = theModule.getTemp(i)-273;
					hue = Math.floor(18240-temp*tmag) % 360;
				}
				ctx.strokeStyle = `hsl(${hue},100%,50%)`;
			} else if ( kind==0 ) { // 0:wood-fixed
				ctx.strokeStyle = "rgb(250,125,0)";
			} else if ( kind==-1 ) { // -1:wood-free
				ctx.strokeStyle = "rgb(200,200,0)";
			}
			ctx.beginPath();
			ctx.arc( xi*sc+xp, (yMax-yi)*sc+yp, hh0*sc, 0, twoPi, false);
			ctx.stroke();
		}
	}

	function drawFlow( ctx, nnp, sc, xp, yp ) {
		const yMax = p_yMax, vmag = 120.0*p_dt*sc, r = p_hh0*sc, twoPi = 2.0*Math.PI, degree = 180.0/Math.PI ;

		for (let i=0; i<nnp; i++) {
			let xi, yi, vxi, vyi;
			[ xi, yi ] = theModule.getPosition(i);
			[ vxi, vyi ] = theModule.getVelocity(i);
			const x = xi*sc+xp, y = (yMax-yi)*sc+yp;
			const hue = Math.floor(Math.atan2(vyi,vxi)*degree+180.0);
			ctx.strokeStyle = `hsl(${hue},100%,50%)`;
			ctx.beginPath();
			ctx.moveTo( x, y );
			ctx.lineTo( x+vxi*vmag, y-vyi*vmag);
			ctx.stroke();

			const kind = theModule.getSPKind(i);
			if ( kind==0 || kind==-1 ) { // 0:wood-fixed, -1:wood-free
				ctx.strokeStyle = (kind==0) ? "rgb(250,125,0)" : "rgb(200,200,0)"
				ctx.beginPath();
				ctx.arc(x, y, r, 0, twoPi, false);
				ctx.stroke();
			}
		}
	}


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setDispMode() {
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function setNcalc() {
		v_nCalc = 1 + document.getElementById("slct_nCalc").selectedIndex;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, p_NNp ];
  }

  function pygetParticlesList() {
		perticleFlag = false;
    return [ nowData, kindList, xxList, yyList, vxList, vyList, densList, pressList, tempList ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		// dom control
		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setDispMode:	setDispMode,	// setDispMode()
		setNcalc:		setNcalc,		// setNcalc()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, nnp ]
    pygetParticlesList, pygetParticlesList, //() :return [ nowData, kindList, xxList, yyList, vxList, vyList, densList, pressList, tempList ]
	};

})(); // ====================  js041 module end  ====================


const js = js041;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js041] wind tunnel - smoothed particle hydrodynamics 2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" width="540" height="360"></canvas>
<br>

<label>theme: wind tunnel</label>
    <span style="margin-right: 150px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label> disp. mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option>smooth particle</option><option selected>density</option>
<option>pressure</option><option>temperature</option>
<option selected>flow</option>
</select>
    <span style="margin-right: 80px;"></span>
<label>speed(nCalc/frame):</label>
<select id="slct_nCalc" onChange="js.setNcalc()">
<option>1</option><option selected>2</option>
<option>3</option><option>4</option>
</select>
<br>

<p id="text_caption" ></p>
<hr width="540" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# get data and print

import time

# simulator run
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  [ sysTime, nnp ] = eval_js( 'js.pygetData({})'.format(i) )
  print(f'i = {i:>2d},  time = {sysTime:>8.3f} (s)')
  time.sleep(3)

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- stop --")

In [ ]:
# change dispMode

import time

dispModeList = [ '0: smooth particle', '1: density', '2: pressuree', '3: temperature', '4: flow' ]

# simulator run
exec_html_js()
print("-- start --")

# change dispMode
for dispMode in [ 0, 1, 2, 3, 4 ]:
  eval_js( 'js.pysetDispMode({})'.format(dispMode) )
  print( "-- dispMode: ", dispModeList[dispMode], "--" )
  [ sysTime, nnp ] = eval_js( 'js.pygetData({})'.format(0) )
  print(f'\t  time = {sysTime:>8.3f}')
  time.sleep(4)

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- stop --")

In [ ]:
# get smoothed particles data

import time
import numpy as np

# simulator run
exec_html_js()
print("-- start --")

# wait wait_time sec
wait_time = 30
print(f"-- wait {wait_time} sec. --")
time.sleep(wait_time)

# get particle dara and plot
print("-- get particles data --")
[ nowData, kindList, xxList, yyList, vxList, vyList, densList, pressList, tempList ] = eval_js( 'js.pygetParticlesList()' )
[ gotTime, nnp ] = nowData
print(f"number of smoothed particle = {nnp} at time ={gotTime:>6.3f}")

# simulator stop
eval_js( 'js.breakLoop()' )
print("-- simulator stop --")

# pack data
packed_data = np.array([ kindList, xxList, yyList, vxList, vyList, densList, pressList, tempList ])
print(f'shape of packed_data :{packed_data.shape}')

# save data
np.save( 'js041_data.npy', packed_data )
print("-- saved --")

In [ ]:
# load data and set numpy array

import numpy as np

x_box_size = 20.0
y_box_size = 10.0

# load data
loaded_data = np.load('js041_data.npy')
print(f'the shape of loaded_data :{loaded_data.shape}')

# set numpy array
Kind = loaded_data[0].astype(int)  # kind of particle  1:air 0:wood-fixed -1:wood-free
X = loaded_data[1]     # x-position of SPH particle
Y = loaded_data[2]     # y-position of SPH particle
U = loaded_data[3]     # x-velocity of SPH particle
V = loaded_data[4]     # x-velocity of SPH particle
Dens = loaded_data[5]  # x-density at the particle position
Press = loaded_data[6] # x-pressure at the particle position
Temp = loaded_data[7]  # x-temperature at the particle position
print(f'the shape of   Kind, X, Y, U, V, Dens, Press, Temp :{X.shape}')

In [ ]:
# plot pressure

import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.title("pressure at every particle")
plt.scatter(X, Y, s=100, c=Press, cmap='jet')
plt.xlabel("x")
plt.ylabel("y")
plt.xlim(0,x_box_size)
plt.ylim(0,y_box_size)
plt.show()

In [ ]:
# pressure(x,y) scatter 3D plot

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 3d scatter plot
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X, Y, Press, s=50, c=Press, cmap='jet')
ax.set_box_aspect((1.0,y_box_size/x_box_size,0.5))
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('Pressure')
plt.title('Pressure(x,y) in 3D')
plt.show()

In [ ]:
# pressure(x,y)scatter 3D plot - plotly

import numpy as np
import plotly.graph_objects as go

# 3d scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=X, y=Y, z=Press,
    mode='markers', marker=dict(size=5, color=Press, colorscale='jet' ))])

fig.update_layout(
    title='Pressure(x,y) in 3D',
    width=1200, height=700,
    scene = dict(
        zaxis_title='Pressure',
        aspectratio=dict(x=1.0, y=y_box_size/x_box_size, z=0.5),
        camera=dict(
            eye=dict(x=0, y=-1.0, z=1.0),  # position of camera
            center=dict(x=0, y=0, z=0),  # camera look at
            up=dict(x=0, y=0, z=1)  # upside vector
    )),)

fig.show()

# save figure as html file
#fig.write_html("js041_pressure.html")

In [ ]:
# flow V(x,y) plot

import numpy as np
import matplotlib.pyplot as plt

# vector field plot
fig = plt.figure(figsize=(12, 6))
ax1 = fig.add_subplot(111)

scale_factor = 1000
ax1.quiver(X, Y, U, V, scale=scale_factor )
plt.title("flow vector (vx,vy)")
plt.show()

In [ ]:
# prompt: in '# flow V(x,y) plot' cell', change color as c=Vabs, cmap = 'jet'

# vector field plot
fig = plt.figure(figsize=(12, 6))
ax1 = fig.add_subplot(111)

scale_factor = 1000
Vabs = np.sqrt(U**2 + V**2)
ax1.quiver(X, Y, U, V, Vabs, scale=scale_factor, cmap='jet' )
plt.title("flow vector (vx,vy)")
plt.show()


In [ ]:
# flow V(x,y) plot ver.2

import numpy as np
import matplotlib.pyplot as plt

# vector field plot
fig = plt.figure(figsize=(12, 6))
ax1 = fig.add_subplot(111)

scale_factor = 1000
Vabs = np.sqrt(U**2 + V**2)
ax1.quiver(X, Y, U, V, Vabs, scale=scale_factor, cmap='jet' )
plt.title("flow vector (vx,vy)")

# Plot the non-air(wood) particles
X_wood = X[Kind != 1]
Y_wood = Y[Kind != 1]
plt.scatter(X_wood, Y_wood, s=30, color='brown')

plt.show()

In [ ]:
# flow(x,y), Press(x,y) - cone plot / plotly

import numpy as np
import plotly.graph_objects as go

W = np.zeros_like(X)

# create quiver plot
fig = go.Figure(data=go.Cone(
    x=X, y=Y, z=Press,
    u=U, v=V, w=W,
    colorscale="Blues",
    sizemode="absolute",
    sizeref=100,
    showscale=False
))

fig.update_layout(
    title='Pflow(x,y) z:Pressure(x,y)',
    width=1200, height=700,
    scene = dict(
        zaxis_title='Pressure',
        aspectratio=dict(x=1.0, y=y_box_size/x_box_size, z=0.5),
        camera=dict(
            eye=dict(x=0, y=-1.0, z=0.5),  # position of camera
            center=dict(x=0, y=0, z=0),  # camera look at
            up=dict(x=0, y=0, z=1)  # upside vector
    )),)

fig.show()

In [ ]:
# plot flow and SPH particles

import numpy as np
import matplotlib.pyplot as plt

# set time span
dt = 0.01  # time span ## change manually 0.001 --> 0.01

# create plot
plt.figure(figsize=(12, 6))
plt.title("Flow")

# plot particle position
plt.scatter(X, Y, s=10, c='blue', alpha=0.5) ## change manually  s=50  --> s=10

# plot flow line
for i in range(len(X)):
  plt.plot([X[i], X[i] + U[i] * dt], [Y[i], Y[i] + V[i] * dt], color='red', alpha=0.5)

# set axis and show plot
plt.xlim(0,x_box_size)
plt.ylim(0,y_box_size)
plt.xlabel("x")
plt.ylabel("y")
plt.show()

In [ ]:
# histogram of Vabs

import numpy as np
import matplotlib.pyplot as plt

Vabs = np.sqrt(U**2 + V**2)
plt.hist(Vabs, 30)

In [ ]:
# corr and scatter matrix of X, Y, U, V, Vabs, Press and Temp

import pandas as pd

# set data flame
df = pd.DataFrame({'X': X, 'Y': Y, 'U': U, 'V': V, 'Vabs': Vabs, 'Press': Press, 'Temp': Temp})

# calc correlation and scatter matrix and print
corr = df.corr()
print('Correlation matrix:')
print(corr)
pd.plotting.scatter_matrix(df)
plt.show()